In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'], os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable, sys.executable

# Import the basic spark library
from pyspark.sql import SparkSession

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("spark_tutorial") \
      .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/06 15:37:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/06 15:37:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/12/06 15:37:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/12/06 15:37:58 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, ArrayType

#Createe the schema using StructField(Name, Type, Nullable)
schema = StructType([ \
    StructField("Pizza Name", StringType(), True), \
    StructField("Price", FloatType(), True), \
    StructField("Ingredients_ID", StringType(), True) \
])

df_data = [("Margherita", 5.95, "IG_1"),
        ("Calzone", 7.95, "IG_2"),
        ("Diavola", 5.95, "IG_3"),
        ("Prosciutto", 7.95, "IG_4"),
        ("Speck & Brie", 7.95, "IG_7"),
        ("Tonno & Cipolle", 7.95, "IG_8")]

df = spark.createDataFrame(data = df_data, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- Pizza Name: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Ingredients_ID: string (nullable = true)



+---------------+-----+--------------+
|Pizza Name     |Price|Ingredients_ID|
+---------------+-----+--------------+
|Margherita     |5.95 |IG_1          |
|Calzone        |7.95 |IG_2          |
|Diavola        |5.95 |IG_3          |
|Prosciutto     |7.95 |IG_4          |
|Speck & Brie   |7.95 |IG_7          |
|Tonno & Cipolle|7.95 |IG_8          |
+---------------+-----+--------------+



In [3]:
ingredient_schema = StructType([ \
    StructField("Ingredients_ID", StringType(), True), \
    StructField("Ingredients", ArrayType(StringType()), True) \
])

ingredient_df_data = [("IG_1", ["Tomato Sauce", "Mozzarella Cheese", "Basil"]),
                    ("IG_2", ["Tomato Sauce", "Mozzarella Cheese", "Prosciutto Cotto"]),
                    ("IG_3", ["Tomato Sauce", "Mozzarella Cheese", "Spicy Salame"]),
                    ("IG_4", ["Tomato Sauce", "Mozzarella Cheese", "Prosciutto Cotto"]),
                    ("IG_5", ["Tomato Sauce", "Mozzarella Cheese", "Speck", "Brie"]),
                    ("IG_6", ["Tomato Sauce", "Mozzarella Cheese", "Tuna", "Onions"])]

ingredient_df = spark.createDataFrame(data = ingredient_df_data, schema = ingredient_schema)
ingredient_df.printSchema()
ingredient_df.show(truncate=False)

root
 |-- Ingredients_ID: string (nullable = true)
 |-- Ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)

+--------------+---------------------------------------------------+
|Ingredients_ID|Ingredients                                        |
+--------------+---------------------------------------------------+
|IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |
|IG_2          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|IG_3          |[Tomato Sauce, Mozzarella Cheese, Spicy Salame]    |
|IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|IG_5          |[Tomato Sauce, Mozzarella Cheese, Speck, Brie]     |
|IG_6          |[Tomato Sauce, Mozzarella Cheese, Tuna, Onions]    |
+--------------+---------------------------------------------------+



<h4>Join Operations</h4>

In [4]:
# Inner join - returns the tuples that matched in both tables
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "inner").show(truncate=False)

+----------+-----+--------------+--------------+---------------------------------------------------+
|Pizza Name|Price|Ingredients_ID|Ingredients_ID|Ingredients                                        |
+----------+-----+--------------+--------------+---------------------------------------------------+
|Margherita|5.95 |IG_1          |IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |
|Calzone   |7.95 |IG_2          |IG_2          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|Diavola   |5.95 |IG_3          |IG_3          |[Tomato Sauce, Mozzarella Cheese, Spicy Salame]    |
|Prosciutto|7.95 |IG_4          |IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
+----------+-----+--------------+--------------+---------------------------------------------------+



In [4]:
# Outer join - returns all the tuples from both tables, if no matches are found, the tuples are returned with null values
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "outer") \
     .show(truncate=False)

# Outer join
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "full") \
     .show(truncate=False)

# Outer join
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "fullouter") \
     .show(truncate=False)

+---------------+-----+--------------+--------------+---------------------------------------------------+
|Pizza Name     |Price|Ingredients_ID|Ingredients_ID|Ingredients                                        |
+---------------+-----+--------------+--------------+---------------------------------------------------+
|Margherita     |5.95 |IG_1          |IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |
|Calzone        |7.95 |IG_2          |IG_2          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|Diavola        |5.95 |IG_3          |IG_3          |[Tomato Sauce, Mozzarella Cheese, Spicy Salame]    |
|Prosciutto     |7.95 |IG_4          |IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|null           |null |null          |IG_5          |[Tomato Sauce, Mozzarella Cheese, Speck, Brie]     |
|null           |null |null          |IG_6          |[Tomato Sauce, Mozzarella Cheese, Tuna, Onions]    |
|Speck & Brie   |7.95 |IG_7          |null    

+---------------+-----+--------------+--------------+---------------------------------------------------+
|Pizza Name     |Price|Ingredients_ID|Ingredients_ID|Ingredients                                        |
+---------------+-----+--------------+--------------+---------------------------------------------------+
|Margherita     |5.95 |IG_1          |IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |
|Calzone        |7.95 |IG_2          |IG_2          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|Diavola        |5.95 |IG_3          |IG_3          |[Tomato Sauce, Mozzarella Cheese, Spicy Salame]    |
|Prosciutto     |7.95 |IG_4          |IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|null           |null |null          |IG_5          |[Tomato Sauce, Mozzarella Cheese, Speck, Brie]     |
|null           |null |null          |IG_6          |[Tomato Sauce, Mozzarella Cheese, Tuna, Onions]    |
|Speck & Brie   |7.95 |IG_7          |null    

In [5]:
# Left join
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "left") \
     .show(truncate=False)

# Left Outer join
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "leftouter") \
     .show(truncate=False)

+---------------+-----+--------------+--------------+---------------------------------------------------+
|Pizza Name     |Price|Ingredients_ID|Ingredients_ID|Ingredients                                        |
+---------------+-----+--------------+--------------+---------------------------------------------------+
|Margherita     |5.95 |IG_1          |IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |
|Prosciutto     |7.95 |IG_4          |IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|Calzone        |7.95 |IG_2          |IG_2          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|Diavola        |5.95 |IG_3          |IG_3          |[Tomato Sauce, Mozzarella Cheese, Spicy Salame]    |
|Tonno & Cipolle|7.95 |IG_8          |null          |null                                               |
|Speck & Brie   |7.95 |IG_7          |null          |null                                               |
+---------------+-----+--------------+--------

In [6]:
# Right join
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "right") \
     .show(truncate=False)

# Right Outer join
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "rightouter") \
     .show(truncate=False)

+----------+-----+--------------+--------------+---------------------------------------------------+
|Pizza Name|Price|Ingredients_ID|Ingredients_ID|Ingredients                                        |
+----------+-----+--------------+--------------+---------------------------------------------------+
|Margherita|5.95 |IG_1          |IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |
|Prosciutto|7.95 |IG_4          |IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|null      |null |null          |IG_5          |[Tomato Sauce, Mozzarella Cheese, Speck, Brie]     |
|Calzone   |7.95 |IG_2          |IG_2          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|Diavola   |5.95 |IG_3          |IG_3          |[Tomato Sauce, Mozzarella Cheese, Spicy Salame]    |
|null      |null |null          |IG_6          |[Tomato Sauce, Mozzarella Cheese, Tuna, Onions]    |
+----------+-----+--------------+--------------+-------------------------------------------

+----------+-----+--------------+--------------+---------------------------------------------------+
|Pizza Name|Price|Ingredients_ID|Ingredients_ID|Ingredients                                        |
+----------+-----+--------------+--------------+---------------------------------------------------+
|Margherita|5.95 |IG_1          |IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |
|Prosciutto|7.95 |IG_4          |IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|null      |null |null          |IG_5          |[Tomato Sauce, Mozzarella Cheese, Speck, Brie]     |
|Calzone   |7.95 |IG_2          |IG_2          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|Diavola   |5.95 |IG_3          |IG_3          |[Tomato Sauce, Mozzarella Cheese, Spicy Salame]    |
|null      |null |null          |IG_6          |[Tomato Sauce, Mozzarella Cheese, Tuna, Onions]    |
+----------+-----+--------------+--------------+-------------------------------------------

In [7]:
# Left Semi join (i.e., left inner join)
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "leftsemi") \
     .show(truncate=False)

+----------+-----+--------------+
|Pizza Name|Price|Ingredients_ID|
+----------+-----+--------------+
|Margherita|5.95 |IG_1          |
|Calzone   |7.95 |IG_2          |
|Diavola   |5.95 |IG_3          |
|Prosciutto|7.95 |IG_4          |
+----------+-----+--------------+



In [9]:
# Left Anti join - returns all the tuples without a match in the other table
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "leftanti") \
     .show(truncate=False)

+---------------+-----+--------------+
|Pizza Name     |Price|Ingredients_ID|
+---------------+-----+--------------+
|Tonno & Cipolle|7.95 |IG_8          |
|Speck & Brie   |7.95 |IG_7          |
+---------------+-----+--------------+



In [20]:
from pyspark.sql.functions import col
# Self join
df.alias("df_1").join(df.alias("df_2"), col("df_1.Ingredients_ID") == col("df_2.Ingredients_ID"), "inner") \
     .show(truncate=False)

+---------------+-----+--------------+---------------+-----+--------------+
|Pizza Name     |Price|Ingredients_ID|Pizza Name     |Price|Ingredients_ID|
+---------------+-----+--------------+---------------+-----+--------------+
|Margherita     |5.95 |IG_1          |Margherita     |5.95 |IG_1          |
|Calzone        |7.95 |IG_2          |Calzone        |7.95 |IG_2          |
|Diavola        |5.95 |IG_3          |Diavola        |5.95 |IG_3          |
|Prosciutto     |7.95 |IG_4          |Prosciutto     |7.95 |IG_4          |
|Speck & Brie   |7.95 |IG_7          |Speck & Brie   |7.95 |IG_7          |
|Tonno & Cipolle|7.95 |IG_8          |Tonno & Cipolle|7.95 |IG_8          |
+---------------+-----+--------------+---------------+-----+--------------+



In [8]:
# It is also possible to concatenate multiple joins one after another
df.join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "inner") \
  .drop(ingredient_df.Ingredients_ID) \
  .join(ingredient_df, df.Ingredients_ID == ingredient_df.Ingredients_ID, "right") \
  .show(truncate=False)

+----------+-----+--------------+---------------------------------------------------+--------------+---------------------------------------------------+
|Pizza Name|Price|Ingredients_ID|Ingredients                                        |Ingredients_ID|Ingredients                                        |
+----------+-----+--------------+---------------------------------------------------+--------------+---------------------------------------------------+
|Margherita|5.95 |IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |IG_1          |[Tomato Sauce, Mozzarella Cheese, Basil]           |
|Prosciutto|7.95 |IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|IG_4          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cotto]|
|null      |null |null          |null                                               |IG_5          |[Tomato Sauce, Mozzarella Cheese, Speck, Brie]     |
|Calzone   |7.95 |IG_2          |[Tomato Sauce, Mozzarella Cheese, Prosciutto Cott